In [10]:
# Note: After you run this cell, the training and test data will be available in
# the file browser. (Click the folder icon on the left to view it)
#
# If you don't see the data after the cell completes, click the refresh button
# in the file browser (folder icon with circular arrow)

# First, let's download and unzip the data
!echo "Downloading files..."
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training1.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/training2.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test.zip
!wget -q https://github.com/byui-cse/cse450-course/raw/master/data/roadsigns/test_classes.csv

!echo "Unzipping files..."
!unzip -q /content/training1.zip
!unzip -q /content/training2.zip
!unzip -q /content/test.zip

# Combine the two traning directories
!echo "Mergining training data..."
!mkdir /content/training
!mv /content/training1/* /content/training
!mv /content/training2/* /content/training

# Cleanup
!echo "Cleaning up..."
!rmdir /content/training1
!rmdir /content/training2
!rm training1.zip
!rm training2.zip
!rm test.zip

!echo "Data ready."

Unzipping files...
Mergining training data...
Cleaning up...
Data ready.


In [11]:
# Import libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

In [12]:
# Create an image training dataset

# We're using keras' image_dataset_from_directory method to load our image data.
# See (https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory) for details
#
# A couple of things to note:
# 1. We're specifying a number for the seed, so we'll always get the same shuffle and split of our images.
# 2. Class names are inferred automatically from the image subdirectory names.
# 3. We're splitting the training data into 80% training, 20% validation. 

training_dir = '/content/training/'
image_size = (100, 100)

# Split up the training data images into training and validations sets
training_data = image_dataset_from_directory(training_dir, validation_split=.2, subset='training', seed=42, image_size=image_size)
validation_data = image_dataset_from_directory(training_dir, validation_split=.2, subset='validation', seed=42, image_size=image_size)

Found 39209 files belonging to 43 classes.
Using 31368 files for training.
Found 39209 files belonging to 43 classes.
Using 7841 files for validation.


In [13]:
df = pd.read_csv(str("/content/test_classes.csv"))
test_data = np.array([np.array(Image.open("/content/test/" + fname)) for fname in df["Filename"]])
test_labels = df["ClassId"]

In [ ]:
# test_labels.dtype

In [ ]:
# # View first 9 images and their class labels
# plt.figure(figsize=(10, 10))
# for images, labels in training_data.take(1):
#   for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
#     plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(training_data.class_names[labels[i]])
#     plt.axis("off")

In [14]:
imported_model = keras.applications.Xception(weights=None, input_shape=(100, 100, 3), include_top=False)

model = tf.keras.Sequential([
  keras.Input(shape=(100, 100, 3)),
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.experimental.preprocessing.RandomTranslation(0.20, 0.20),
  layers.experimental.preprocessing.RandomZoom(0.20, 0.20),
  layers.experimental.preprocessing.RandomRotation(0.07),
  layers.experimental.preprocessing.RandomContrast(0.25),

  imported_model,

  layers.GlobalAveragePooling2D(),
  layers.Dense(43)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
model.fit(training_data, epochs=7, validation_data=validation_data)

Epoch 1/7
981/981 [==============================] - 173s 138ms/step - loss: 3.9310 - accuracy: 0.0839 - val_loss: 11.4594 - val_accuracy: 0.0661
Epoch 2/7
981/981 [==============================] - 134s 137ms/step - loss: 2.6150 - accuracy: 0.2215 - val_loss: 2.6000 - val_accuracy: 0.3521
Epoch 3/7
981/981 [==============================] - 134s 136ms/step - loss: 1.4938 - accuracy: 0.4942 - val_loss: 1.3392 - val_accuracy: 0.6293
Epoch 4/7
981/981 [==============================] - 134s 136ms/step - loss: 0.6022 - accuracy: 0.7996 - val_loss: 0.3060 - val_accuracy: 0.8981
Epoch 5/7
981/981 [==============================] - 134s 136ms/step - loss: 0.2497 - accuracy: 0.9190 - val_loss: 0.1778 - val_accuracy: 0.9393
Epoch 6/7
981/981 [==============================] - 133s 136ms/step - loss: 0.1595 - accuracy: 0.9485 - val_loss: 0.0654 - val_accuracy: 0.9786
Epoch 7/7
981/981 [==============================] - 133s 136ms/step - loss: 0.1205 - accuracy: 0.9622 - val_loss: 0.0797 - val_a

In [16]:


print("Test Results:")
test_loss, test_acc = model.evaluate(test_data, test_labels.astype("uint8"), verbose=2)
y = model.predict(test_data)


Test Results:
395/395 - 11s - loss: 0.2745 - accuracy: 0.9338


In [34]:
def findF1(cm):
  tp = 0
  fp = 0
  fn = 0
  for i in range(43):
    for j in range(43):  
      if (i == j):
        tp += cm[i][j]
      else:
        fp += cm[j][i] 
        fn += cm[i][j]
  tp = tp / 12630
  fp = fp / 12630
  fn = fn / 12630
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  return 2 * ((precision * recall) / (precision + recall))

In [38]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
listAcc = list()
y_true = test_labels.to_list()
y_pred = np.argmax(y, axis=1)
cm = confusion_matrix(y_true, y_pred)
for i in range(43):
  total = 0
  right = 0
  for j in range(43):
    if (i == j):
      right += cm[i][j]
    total += cm[i][j]
  listAcc.append((right / total))
exact = 0
for x in range(43):
  if listAcc[x] > .95:
    exact += 1
print(exact)
print(cm)
print(y.shape)
print(listAcc)
print(findF1(cm))
np.savetxt("foo.csv", cm, delimiter=',', fmt = "%i")


21
[[ 40   0   0 ...   0   0   0]
 [  0 688   0 ...   0   0   0]
 [  0   1 681 ...   5   0   0]
 ...
 [  0   0   0 ...  83   0   0]
 [  0   0   0 ...   0  59   0]
 [  0   0   0 ...   0   0  89]]
(12630, 43)
[0.6666666666666666, 0.9555555555555556, 0.908, 0.9155555555555556, 0.8545454545454545, 0.9904761904761905, 0.52, 0.9266666666666666, 0.9888888888888889, 0.9979166666666667, 0.9984848484848485, 0.9380952380952381, 0.8681159420289855, 0.9958333333333333, 1.0, 0.9952380952380953, 1.0, 0.9666666666666667, 0.9384615384615385, 0.9333333333333333, 0.9888888888888889, 0.6, 0.9416666666666667, 0.8733333333333333, 0.7666666666666667, 0.9729166666666667, 0.9277777777777778, 0.4, 0.74, 1.0, 0.78, 0.9481481481481482, 0.7333333333333333, 0.9761904761904762, 0.9833333333333333, 0.9564102564102565, 0.8833333333333333, 0.9833333333333333, 0.9956521739130435, 0.9666666666666667, 0.9222222222222223, 0.9833333333333333, 0.9888888888888889]
0.9338083927157561
